In [4]:
import matplotlib.pyplot as plt
import numpy as np
from ase.io import write
from ase.build import nanotube
from ase.neighborlist import NeighborList
from scipy.linalg import eigh
from ase.visualize import view


In [5]:
def build_device_structure(n=6, m=0, N_L=3, N_M=4, N_R=3, bond=1.42, direction='x', tag_regions=True):
    """
    Build a CNT device using ASE nanotube with clear region tagging.

    Args:
        n, m: Chiral indices.
        N_L, N_M, N_R: Unit cells in left, middle, right.
        bond: C–C bond length.
        direction: Axis of tube ('x', 'y', or 'z').
        tag_regions: Whether to tag atoms by region (L=1, M=2, R=3).

    Returns:
        ase.Atoms with 'region' tags if enabled.
    """
    total_cells = N_L + N_M + N_R
    atoms = nanotube(n=n, m=m, length=total_cells, bond=bond, verbose=False)

    # Rotate so tube lies along 'direction'
    if direction == 'x':
        atoms.rotate('z', 'x', rotate_cell=True, center='COP')
        axis = 0
    elif direction == 'y':
        atoms.rotate('z', 'y', rotate_cell=True, center='COP')
        axis = 1
    elif direction == 'z':
        axis = 2
    else:
        raise ValueError("Direction must be 'x', 'y', or 'z'")

    # Determine boundaries
    positions = atoms.get_positions()
    coord = positions[:, axis]
    x_min, x_max = coord.min(), coord.max()
    total_len = x_max - x_min
    uc_len = total_len / total_cells

    # Region boundaries
    x_L = x_min + N_L * uc_len
    x_M = x_L + N_M * uc_len

    if tag_regions:
        regions = []
        for x in coord:
            if x < x_L:
                regions.append(1)  # Left
            elif x < x_M:
                regions.append(2)  # Middle
            else:
                regions.append(3)  # Right
        atoms.set_array('region', np.array(regions, dtype=int))

    atoms.set_pbc((1, 0, 0))  # set periodic only in tube direction
    return atoms

In [10]:
def apply_strain(atoms, strain_percent, axis=0):
    """
    Apply uniaxial strain only to region 2 (middle) of the CNT structure,
    anchoring the left boundary and shifting region 3 to preserve bond lengths.

    Args:
        atoms: ASE Atoms object with 'region' array.
        strain_percent: Strain percentage to apply.
        axis: Axis along which to apply strain (default: 0 for x-axis).

    Returns:
        ASE Atoms object with region 2 strained (anchored on left) and region 3 shifted.
    """
    if 'region' not in atoms.arrays:
        raise ValueError("Atoms object must have 'region' array to apply region-specific strain.")

    positions = atoms.get_positions()
    regions = atoms.get_array('region')

    strain = 1 + strain_percent / 100.0
    coord = positions[:, axis]
    mask_M = regions == 2
    mask_R = regions == 3

    # Left anchor point of region 2
    x_M_min = coord[mask_M].min()
    x_M_max = coord[mask_M].max()
    original_length = x_M_max - x_M_min

    # Stretch region 2 from the left boundary
    strained_positions = positions.copy()
    strained_positions[mask_M, axis] = x_M_min + strain * (positions[mask_M, axis] - x_M_min)

    # Shift region 3 to preserve bond lengths
    delta = (strain - 1.0) * original_length
    strained_positions[mask_R, axis] += delta

    atoms.set_positions(strained_positions)
    return atoms


In [11]:
# Parameters
n, m = 6, 0
ncells = 2
bond = 1.42
onsite = 0.0  # eV
first_neighbor_hopping = -2.7  # eV
beta = 0 #3.37
neighbor_cutoff = 1.43  # Å for nearest neighbors
d0 = bond


In [12]:
device = build_device_structure(n=6, m=0, N_L=1, N_M=28, N_R=1, direction='x')
write("device_init.xyz", device)
device = apply_strain(device, 2.0, axis=0)
write("device_strained.xyz", device)